In [1]:
import numpy as np
import sys
#np.set_printoptions(precision=5, linewidth=200, suppress=True)
import psi4
from helper_PFCI import PFHamiltonianGenerator
from helper_PFCI import Determinant
from helper_cqed_rhf import cqed_rhf
np.set_printoptions(threshold=sys.maxsize)

In [2]:
mol_str = """
0 1
o
h 1 1.0
h 1 1.0 2 104.0
symmetry c1
"""

options_dict = {'basis': '6-31g',
                  'scf_type': 'pk',
                  'e_convergence': 1e-10,
                  'd_convergence': 1e-10
                  }

mol = psi4.geometry(mol_str)


psi4.set_options(options_dict)
psi4.core.set_output_file('output.dat', False)

In [3]:
# photon energy
#omega_val = 2.0 / psi4.constants.Hartree_energy_in_eV
omega_val = 0

print(omega_val)

# lambda vector
lambda_vector = np.array([0., 0., 0.0])
#lambda_vector = np.array([0.5, 0.5, 0.5])

n_act_orb=5
#number of active electrons
n_act_el=4
# Build PF Hamiltonian matrix 
H2_PF = PFHamiltonianGenerator(1, mol_str, options_dict, lambda_vector,omega_val, n_act_el,n_act_orb,ignore_coupling=True, cas=False)


0

Start SCF iterations:

Canonical RHF One-electron energy = -122.2743570184695727
CQED-RHF One-electron energy      = -122.2743570184695727
Nuclear repulsion energy          = 8.8026031303907679
Dipole energy                     = 0.0000000000000000
SCF Iteration   1: Energy = -75.9800163449004344   dE = -7.59800E+01   dRMS = 5.74391E-13
SCF Iteration   2: Energy = -75.9800163449003350   dE =  9.94760E-14   dRMS = 1.52547E-13
Total time for SCF iterations: 0.001 seconds 

QED-RHF   energy: -75.98001634 hartree
Psi4  SCF energy: -75.98001634 hartree


In [4]:
# diagonalize the Hamiltonian matrix
e_fci, wavefunctions = np.linalg.eigh(H2_PF.H_PF)


In [5]:
H2_PF.calc1RDMfromCIS(wavefunctions[:,14])
print(np.trace(H2_PF.Dij))
print(np.trace(H2_PF.Dab))
print(np.trace(H2_PF.D1))



e1_test_rdm = np.einsum("pq,pq->", H2_PF.Hspin, H2_PF.D1)

temp=np.einsum("pq,q->p", H2_PF.H_1E,wavefunctions[:,14])
e1_test_wfn = np.dot(wavefunctions[:,14].T,temp)




9.000000000000007
1.0000000000000007
10.000000000000007


In [6]:
print(e1_test_rdm)
print(e1_test_wfn-H2_PF.Enuc)
print(e1_test_rdm-e1_test_wfn+H2_PF.Enuc)

-119.48756194249603
-119.48756194249606
3.197442310920451e-14
